In [1]:
import torchvision
import numpy as np
import time
from fastai.vision import *

#!/usr/bin/env python
# import rospy

import sys

try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2 as cv
import torch
from torch.autograd.variable import Variable
from torchvision.transforms import Normalize
import sys
import numpy as np
from PIL import Image

imagenet_stats = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# try:
#     sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')   # It causes cv2 import error
# except:
#     print('yeah!')


def preprocess(images):
    images = torch.unsqueeze(torch.from_numpy(images),dim=0)
    images = images.float()
    images = Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(images)
    return images#.half()

def get_subtracted(images):
    images = images.view(images.size(0)/2, 2, 3, 480, 640)
    images1, images2 = images[:,0], images[:,1]
    return images1-images2

def get_masked(img, mask):
    masked_image = np.multiply(img, mask/255)
    return masked_image

def getnp(mat):
    mat = mat.cpu()
    return mat.detach().numpy()

def get_control(msg):
    t1 = time.time()
    frame = cv.imdecode(np.frombuffer(msg.data, dtype=np.uint8), 1)
    cv.imwrite('buffer.jpg', frame)
    test_image = open_image('buffer.jpg')
    img_segment = seg_learn.predict(test_image)[0]

    masked = np.multiply(test_image.data, img_segment.data)
    torchvision.utils.save_image(masked, 'bh_test_masked.jpg')

    masked = open_image('bh_test_masked.jpg')
    action = bh_learn.predict(masked)

    # publish action on a node
#     pub = rospy.Publisher('/in_put', Int32 , queue_size=1)
#     try:
#         print(int(action[0]))
#         a = pub.publish(int(action[0]))
#         print(a)
#     except:
#         print('could not publish')
#     print(time.time() - t1)



# if __name__ == '__main__':
    #fastai models
seg_learner_path = './models/segmentation_model/'
bh_learner_path = './models/bh_cloning_model/'
seg_learn = load_learner(seg_learner_path).to_fp32()
bh_learn = load_learner(bh_learner_path).to_fp32()

#     print('Press Ctrl+C for exiting')
#     rospy.init_node('detector', anonymous=True)
#     rospy.Subscriber("/output/image_raw/compressed", CompressedImage, get_control)
#     rospy.spin()


In [2]:
seg_learn.summary

<bound method model_summary of Learner(data=ImageDataBunch;

Train: LabelList (0 items)
x: SegmentationItemList

y: SegmentationLabelList

Path: models/segmentation_model;

Valid: LabelList (0 items)
x: SegmentationItemList

y: SegmentationLabelList

Path: models/segmentation_model;

Test: None, model=DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

In [9]:
# bh_learn.summary

In [10]:
# frame = cv.imread('day1.jpg')
# cv.imwrite('day1.jpg', frame)
test_image = open_image('day1_res.jpg')
img_segment = seg_learn.predict(test_image)[0]

masked = np.multiply(test_image.data, img_segment.data)
torchvision.utils.save_image(masked, 'bh_test_masked.jpg')
torchvision.utils.save_image(masked*255, 'bh_masked.jpg')

masked = open_image('bh_test_masked.jpg')
action = bh_learn.predict(masked)

In [4]:
test_image_.shape

(780, 1040, 3)

In [5]:
a = cv.resize(test_image_,(640,480))

In [6]:
a.shape

(480, 640, 3)

torch.Size([1, 480, 640])

In [10]:
import cv2 as cv
import numpy as np
import argparse
import os
import json


def disp_polygon_mask(image = None, Mask = None):
    alpha = 0.5

    try:
        image = args.image
        mask = args.mask
        print('--------------------------------')

    except:
        pass

    image = cv.imread(image)
    mask = np.load(Mask)

    if image is None:
        print("Error loading src1")
        exit(-1)
    elif mask is None:
        print("Error loading src2")
        exit(-1)
    mask[:,:,1] = 0;
    seg = cv.addWeighted(image, alpha, mask, (1-alpha), 0.0)
    cv.imshow('Segmented', seg)
    cv.waitKey(0)
    cv.destroyAllWindows()


def get_mask():

    if not os.path.exists(args.directory):
        os.mkdir(args.directory)
        print('Created out_dir')

    for name in os.listdir(args.input_image):
        frame = cv.imread(args.input_image + '/' + name)
        with open(str('./' + args.input_annotations + '/' + name.strip('.jpg') + '.json')) as json_file:
            data = json.load(json_file)
        key_points = data['shapes'][0]['points']
        mask = np.zeros(frame.shape, np.uint8)
        cv.fillPoly(mask, np.int32([np.array(key_points)]), color = (255,255,255))
        # cv.imshow('temp',mask)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        cv.imwrite(args.directory + '/' + name, mask)

def get_masked(img, mask):

    masked_image = np.multiply(img, mask/255)
    return masked_image

In [11]:
# ! ls data/stairtestset/adversaries/

In [21]:
mask1 = cv.imread('data/stairtestset/adversaries/mask1.jpg')
mask2 = cv.imread('data/stairtestset/adversaries/mask2.jpg')

In [32]:
cv.imshow('mask1',mask2*255)
cv.waitKey(0)
cv.destroyAllWindows()

In [143]:
img1 = cv.imread('data/stairtestset/adversaries/1.jpg')
img1 = cv.resize(img1,(640,480))
img2 = cv.imread('data/stairtestset/adversaries/2.jpg')
img2 = cv.resize(img2,(640,480))
this2 = get_masked(img2,mask2)

In [144]:
cv.imshow('mask1',this2)
cv.waitKey(0)
cv.destroyAllWindows()

In [145]:
cv.imwrite('data/stairtestset/adversaries/masked_pt2.jpg',this2*255)

True

In [132]:
# Get real masked images
ame = 'night3'

In [133]:
img = cv.imread('data/stairtestset/'+str(ame)+'.jpg')
with open(str('data/stairtestset/'+str(ame)+'.json')) as json_file:
            data = json.load(json_file)
key_points = data['shapes'][0]['points']
mask = np.zeros((780, 1040, 3), np.uint8)
mask = cv.fillPoly(mask, np.int32([np.array(key_points)]), color = (1,1,1))

In [134]:
masked = masked_image = np.multiply(img,mask)
m = cv.resize(masked,(640, 480))
cv.imwrite('data/stairtestset/'+str(ame)+'_gt.jpg',m)

True

In [135]:
# cv.imshow('sthi', masked)
# cv.waitKey(0)
# cv.destroyAllWindows()
test_image = open_image('data/stairtestset/'+str(ame)+'_res.jpg')
img_segment = seg_learn.predict(test_image)[0]
masked_passed = np.multiply(test_image.data, img_segment.data)
torchvision.utils.save_image(masked_passed, 'data/stairtestset/'+str(ame)+'_pt.jpg')

(480, 640, 3)

In [12]:
img = cv.imread('data/stairtestset/day1.jpg')
img.shape

(780, 1040, 3)

In [150]:
# img1 = cv.imread('data/stairtestset/adversaries/1.jpg')
# img1 = cv.resize(img1,(640,480))


In [152]:
cv.imshow('sthi', m)
cv.waitKey(0)
cv.destroyAllWindows()

In [130]:
img1.shape

(480, 640, 3)

In [6]:
imgk.shape

NameError: name 'imgk' is not defined

True

In [188]:
test_image = open_image('data/stairtestset/night3_res.jpg')
img_segment = seg_learn.predict(test_image)[0]
masked = np.multiply(test_image.data, img_segment.data)
torchvision.utils.save_image(masked, 'data/stairtestset/night3_pt.jpg')

In [181]:
test_image.shape

torch.Size([3, 780, 1040])

In [162]:
img_segment.shape

torch.Size([1, 480, 640])

In [174]:
cv.resize(test_image.data.detach().numpy(),(640, 480))

error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/resize.cpp:3427: error: (-215:Assertion failed) !dsize.empty() in function 'resize'


In [2]:
# test_image.data.detach().numpy())
import cv2 as cv

In [28]:
temp = cv.imread('data/stairtestset/night2.jpg')
temp = cv.resize(temp,(640, 480))
cv.imwrite('data/stairtestset/night2_res.jpg',temp)

True